# Inception_v3 AKA GoogleNet_v3

Load the initial pre trained model from pytorch

In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 281MB/s] 


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

Download an example image of a dog



In [12]:
import urllib
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# url, filename = ("https://upload.wikimedia.org/wikipedia/commons/a/a5/Red_Kitten_01.jpg", "kitten.jpg")
url, filename = ("https://t4.ftcdn.net/jpg/01/34/80/63/360_F_134806348_RAkkYc6pfmooITxgFr38x7KxJbdiL0Jc.jpg", "bench.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

Transform the image as needed for the model

*All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 299. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]*

In [13]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

Configure and run the model, print the probabilities

In [15]:
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
  output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-2.2286e-01, -2.5160e-01, -8.2329e-01, -6.5092e-01, -4.2813e-01,
        -8.0414e-02, -5.6310e-01, -6.6466e-01,  3.5194e-01,  8.8382e-01,
         5.9375e-01, -1.9122e-01,  1.2077e+00,  1.0320e+00,  9.9856e-01,
         9.8332e-01,  6.4366e-01,  1.3351e+00,  1.7701e+00,  2.0908e+00,
         2.0633e-01, -1.6273e-01,  3.3008e-01, -4.4310e-01,  5.9225e-01,
        -4.4012e-01, -7.1152e-01, -2.7666e-01, -3.8010e-01,  3.6336e-02,
        -7.5098e-01, -4.6607e-01, -7.0291e-01, -7.8169e-01, -9.9536e-01,
        -4.8212e-01, -6.6985e-01,  6.1725e-02,  7.3151e-02, -8.4547e-01,
        -9.8003e-02, -1.6121e-01,  7.6920e-01,  3.7757e-01, -3.8619e-01,
        -6.8090e-02,  1.4328e-01, -2.6535e-01, -6.7404e-01, -4.3451e-01,
         1.7930e-01, -1.3683e+00,  5.7873e-01,  2.6981e-02, -7.6406e-01,
         1.2131e-02, -2.0300e-01, -8.2518e-01, -2.7597e-01,  1.7678e-01,
         6.3738e-01, -2.3725e-01, -1.4246e+00,  4.6563e-01, -3.8134e-02,
         3.0701e-01,  4.0699e-01,  1.0471e+00,  1.3

Transform probabilities into actual labels

In [16]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

--2024-07-22 09:15:17--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.2’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-07-22 09:15:17 (125 MB/s) - ‘imagenet_classes.txt.2’ saved [10472/10472]

park bench 0.918576717376709
ashcan 0.009436696767807007
lakeside 0.0005912994965910912
chickadee 0.0004747261991724372
spatula 0.00047370907850563526
